In [ ]:
import ollama
import time
from IPython.display import display, clear_output, Markdown
from Qdrant import search_peraturan

In [ ]:
# "messages" berguna untuk mempertahankan konteks"
# Fungsi untuk menghasilkan respon dari model Llama3 menggunakan Ollama
def generate_response_with_ollama(contexts, query):
    context_text = " ".join(contexts)
    input_text = f"""
    Anda adalah chatbot untuk karyawan bertanya mengenai informasi dan peraturan perusahaan.
    Teks informasi dibawah ini tidak dibaca oleh karyawan, tidak perlu menjawab dengan memberitahu BAB maupun subabnya.
    Jika teks informasi yang diberikan tidak ada atau kosong, maka anda jawab "tidak tahu", jangan mengarang jawaban.
    Berikut adalah teks informasi untuk menjawab pertanyaan nantinya:
    
    {context_text}
    
    setelah anda membaca teks informasi tersebut, jawablah pertanyaan berikut dengan jelas sesuai pertanyaannya:
    Jika teks informasi yang diberikan tidak ada atau kosong, maka anda jawab "tidak tahu", jangan mengarang jawaban.
    Pertanyaan: {query}
    
    Jika Anda dapat menemukan jawaban berdasarkan teks informasi tersebut, berikan jawaban yang jelas, tidak perlu menambahkan jawaban tambahan lainnya, dan jangan menambahkan kalimat seperti "Saya memiliki jawaban berdasarkan informasi yang tersedia." 
    Jika Anda tidak dapat menemukan jawaban yang tepat, katakan bahwa "Saya tidak memiliki jawaban berdasarkan informasi yang tersedia" dan "informasi yang saya miliki hanya ..." lalu berikan saran untuk bertanya kepada atasan dan tawarkan bantuan lebih lanjut jika ada pertanyaan lainnya.
    Jawablah dengan bahasa indonesia.
    
    
    """
    
    response = ollama.chat(
        model="llama3.1",
        messages=[{"role": "system","content": "Jawablah pertanyaan berdasarkan informasi berikut. Jika tidak ada informasi yang relevan, katakan 'Informasi yang diminta tidak tersedia."},{"role": "user", "content": input_text},],
        stream=True
    )
    
    return response, context_text  # Mengembalikan generator

# Contoh penggunaan
query = "Apa itu Nominasi?"
results = search_peraturan(query)
response, context_text = generate_response_with_ollama(results, query)

output = ""
for chunk in response:
    output += chunk['message']['content']
    clear_output(wait=True)  # Menghapus output sebelumnya
    display(Markdown(output))  # Menampilkan output yang diperbarui
    time.sleep(0.1)  # Jeda untuk mensimulasikan efek streaming

In [3]:
import requests
from sentence_transformers import SentenceTransformer
import qdrant_client
import torch
from ollama import Client

# Inisialisasi model dan klien
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device='cuda' if torch.cuda.is_available() else 'cpu')
client_qdrant = qdrant_client.QdrantClient("http://10.12.9.105:6333")
ollama_client = Client(host='http://10.12.9.105:11434')

def get_embeddings(text):
    return model.encode(text)

def search_peraturan(query):
    query_vector = get_embeddings(query)
    results = client_qdrant.search(
        collection_name="peraturan_perusahaan",
        query_vector=query_vector,
        with_vectors=True,
        with_payload=True
    )
    return [res.payload["text"] for res in results]

def generate_response_with_ollama(contexts, query):
    context_text = " ".join(contexts)
    input_text = f"""
    Anda adalah chatbot untuk seseorang bertanya mengenai informasi yang telah disediakan di Database Qdrant.
    Teks informasi di bawah ini adalah hasil pencarian yang relevan dengan pertanyaan yang diajukan, namun mungkin tidak tersusun dengan baik.
    Tugas Anda adalah menyusun ulang informasi ini secara koheren dan menjawab pertanyaan dengan lengkap dan jelas.
    
    {context_text}
    
    setelah Anda membaca teks informasi tersebut, jawablah pertanyaan berikut dengan lengkap sesuai pertanyaannya:
    Jika teks informasi yang diberikan tidak ada atau kosong, maka Anda jawab "tidak tahu", jangan mengarang jawaban.
    Pertanyaan: {query}
    
    Jika Anda dapat menemukan jawaban berdasarkan teks informasi tersebut, berikan jawaban yang lengkap dengan susunan yang jelas dan koheren, tidak perlu menambahkan jawaban tambahan lainnya.
    Jika Anda tidak dapat menemukan jawaban yang tepat, katakan bahwa "Saya tidak memiliki jawaban berdasarkan informasi yang tersedia".
    Jawablah sesuai dengan Bahasa yang digunakan di query.
    """

    try:
        response = ollama_client.chat(
            model="llama3.1",
            messages=[
                {"role": "assistant", "content": "Susun ulang informasi berikut secara koheren dan jawab pertanyaan berdasarkan informasi yang tersedia."},
                {"role": "user", "content": input_text}
            ],
            stream=True
        )
        
        output = ""
        for chunk in response:
            output += chunk.get('message', {}).get('content', '')

        return output

    except Exception as e:
        print(f"Error: {e}")
        return "An error occurred while generating the response.", ""

query = "Apa itu anak perusahaan?"
results = search_peraturan(query)
response = generate_response_with_ollama(results, query)

print("Response:", response)

Response: Dalam laporan keuangan Perusahaan (PT Dayamitra Telekomunikasi Tbk), definisi Anak Perusahaan adalah badan usaha atau entitas yang sahamnya sebahagian besar dimiliki oleh Perseroan secara langsung atau yang terkonsolidasi dalam laporan keuangan Perseroan. Selain itu, Direksi dan Dewan Komisaris anak perusahaan juga harus merupakan pengurus badan usaha atau entitas yang sahamnya sebahagian besar dimiliki oleh Perseroan secara langsung atau yang terkonsolidasi dalam laporan keuangan Perseroan.
